In [ ]:
#runModel: executes the model that's been built

In [ ]:
import sys
import os
sys.path.append(os.getcwd()[0:os.getcwd().rfind('/')]+'/bin')

In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats
import argparse

from modules.RunSPARCED import RunSPARCED

In [4]:
# SBML model we want to import
sbml_file = 'SPARCED.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name
# used for filename to save
cellNumber = 0  

In [ ]:
# deterministic=1, stochastic=0
flagD = 1

# deterministic='GrowthStim_det_', stochastic='GrowthStim_stoc_'
nmxlsfile = 'GrowthStim_det_'

ts = 30
th = 2
Vn = 1.75E-12
Vc = 5.25E-12
STIMligs = [100,100.0,100.0,100.0,100.0,100.0,1721.0] # EGF, Her, HGF, PDGF, FGF, IGF, INS

In [ ]:
sys.path.insert(0, os.path.abspath(model_output_dir))
species_sheet = np.array([np.array(line.strip().split("\t")) for line in open('Species.txt', encoding='latin-1')])

species_initializations = []
for row in species_sheet[1:]:
    species_initializations.append(float(row[2]))
species_initializations = np.array(species_initializations)
species_initializations[np.argwhere(species_initializations <= 1e-6)] = 0.0
species_initializations[155:162] = STIMligs

In [ ]:
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver()          # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

xoutS_all, xoutG_all, tout_all = RunSPARCED(flagD,th,species_initializations,[],Vn,Vc,model)

In [ ]:
# saves output
columnsS=[ele for ele in model.getStateIds()]
columnsG = columnsS[773:914]
resa = [sub.replace('m_', 'ag_') for sub in columnsG]
resi = [sub.replace('m_', 'ig_') for sub in columnsG]
columnsG2 = np.concatenate((resa, resi), axis=None)
condsSDF = pd.DataFrame(data=xoutS_all,columns=columnsS)
condsSDF.to_csv(nmxlsfile+'S_'+str(cellNumber)+'.txt',sep="\t")  
condsSDF = None
condsGDF = pd.DataFrame(data=xoutG_all,columns=columnsG2)
condsGDF.to_csv(nmxlsfile+'G_'+str(cellNumber)+'.txt',sep="\t") 
condsGDF = None
cellNumber+=1